## Experimenting with graphframe and pyspark

I tend to follow the User guide for Graphframe and that is posted on Databricks https://docs.databricks.com/spark/latest/graph-analysis/graphframes/user-guide-python.html. 

In [1]:
from src.GHnet import *
from src.parse_edges import *

Now, read the fork edges and the contribution edges from the text file `fork_edges.txt` and `contrib_edges.txt`. For each line in the two files, the edges will be in the form of a tuple `(u,r)` or `(r,u)`. Here `r` and `u` specifies the repositories ID and user ID on GitHub. For more information about the storage formats of `fork_edges.txt` and `contrib_edges.txt`. and how edges are collected (and how the network is formed), please consult the meta file `meta.md`. 

After we read in the contribution edges and fork edges, we will build the Biparitite Graph using Graphframe. Now, we will convert the text file that contains the edges into two parquet file `vertices.parquet` and `edges.parquet`. 

In [2]:
txt_to_pq(contrib_edge_file='data/contrib_edges.txt',
          fork_edge_file='data/fork_edges.txt')

Next, using these two parquet file, we will construct a graphframe object. 

In [2]:
demo = GHnet(vertices_pq='data/vertices.parquet',
            edges_pq='data/edges.parquet')

## Preliminary EDA on the Graph
After we create a graphframe object, we can now do some preliminary EDA on the graph. One straightfoward thing we can check is the in and out degree distribution of the users and repos. 

In [3]:
demo.degree_dist(deg_type='f', plot = True)

NameError: name 'np' is not defined

## Page Rank of GitHub's repository and users
Top ten influential repository?

## Graph Spectrum
Since we don't have the direct collaboration information between users (which requires push access to specific repo), we are only able to collect the contributors of specific repos and users who fork the repos. Thus the contribution-fork edge network is bipartite.

Let $A$  be the Adjacency matrix of GitHub's contribution-fork network and assume there are $m$ repos and $n$ users. $A_{ur}$ be the adjacency matrix that represents contribution edges, and $A_{ru}$ be the adjacency matrix that represents the fork edges. Then $A$ can be written in block matrix form $$A = \begin{pmatrix} 0_n & A_{ur}\\ A_{ru}& 0_m\end{pmatrix}$$. Thus the matrix $A$ is singular, thus having 0 eigenvalue. 

## Community Detection
Implement community detection algorithms

# (If Time allows) Matrix completion problem
We now observe a bipartite contribution-fork network. But we know that users on GitHub collaborate with each other (or they can follow someone they look up to) and repos depends on other repos as well (unless developers enjoy torturing themselves or tired of being tortured by shitty packages they use on a daily basis). We can think of this as a matrix completion problem, where we want to fill in the unobserved entries in our population adjacency matrix. Suppose we have a small sample of collaboration/follower and repository dependency information, can we use that to complete our population graph?